In [1]:
import os
import pyspark
from pyspark.sql import SparkSession, functions as F
import matplotlib.pyplot as plt

os.environ["SPARK_HOME"] = pyspark.__path__[0]
os.environ["PYSPARK_DRIVER_PYTHON"] = "python"
os.environ["PYSPARK_PYTHON"] = "python"

spark = (
    SparkSession.builder
    .config("spark.driver.memory", "8g")
    .getOrCreate()
)

In [5]:
levels = spark.read.json("../data/all_levels_20210801.json", multiLine=True).cache()
ranking = spark.read.json("../data/all_ranking_20210731_deduped.json", multiLine=True).cache()
levels.createOrReplaceTempView("levels")
ranking.createOrReplaceTempView("ranking")
levels.printSchema()
ranking.printSchema()

root
 |-- level: long (nullable = true)
 |-- name: string (nullable = true)
 |-- timestamp: string (nullable = true)

root
 |-- fame: long (nullable = true)
 |-- guild: string (nullable = true)
 |-- job: string (nullable = true)
 |-- level: long (nullable = true)
 |-- mastery: string (nullable = true)
 |-- name: string (nullable = true)
 |-- rank: long (nullable = true)
 |-- specialization: string (nullable = true)



In [8]:
df = spark.sql("""
with names as (
    select distinct name
    from levels
)
select
    name,
    level,
    job,
    specialization,
    mastery,
    fame,
    guild,
    row_number() over (order by level desc, rank) as rank
from ranking
right join names
using (name)
""")

In [11]:
df.count(), spark.sql("select distinct name from levels").count()

(201499, 201499)

In [12]:
df.toPandas().to_csv("../data/20210801_all_ranking.csv", header=True, index=False)

In [13]:
levels.toPandas().to_csv("../data/20210801_all_levels.csv", header=True, index=False)